# How does SuSiE perform given T=200 non-zero effects?

Assume using prior = 0.2. We investigate SuSiE performance given 200 non-zero effects.

## Results
**- Summary: SuSiE power is pretty low given 200 non-zero effects.**

In [6]:
dscout.summary

effect_num,pve,mean_corX,power,fdr,cs_size,cs_num,top_hit_rate,avg_purity
200,0.01,0.1812256,0.0000,0.0000,0.000000,0.000000,0.0000,0.0000000
200,0.02,0.1812256,0.0001,0.5000,1.000000,1.000000,0.5000,1.0000000
200,0.03,0.1812256,0.0004,0.2000,1.000000,1.000000,0.8000,1.0000000
200,0.05,0.1812256,0.0013,0.0714,1.000000,1.000000,0.9286,1.0000000
200,0.10,0.1812256,0.0038,0.0256,1.067568,1.054054,0.9231,0.9940209
200,0.20,0.1812256,0.0066,0.0704,1.020833,1.479167,0.9014,0.9973199
200,0.40,0.1812256,0.0091,0.0714,1.030000,1.960000,0.9082,0.9987936
200,0.50,0.1812256,0.0101,0.0982,1.000000,2.240000,0.8839,0.9990225
200,0.70,0.1812256,0.0133,0.0952,1.000000,2.940000,0.8912,0.9997994
200,0.90,0.1812256,0.0191,0.0991,1.000000,4.240000,0.8915,0.9974634


## Code details

In [1]:
dscout_Q3 = readRDS('dscout_Q3.rds')
dscout_Q3 = dscout_Q3[!is.na(dscout_Q3$sim_gaussian_large.output.file),]
dscout_Q3 = dscout_Q3[!is.na(dscout_Q3$susie_large.output.file),]

In [2]:
dscout_df = data.frame(dscout_Q3$sim_gaussian_large.effect_num, dscout_Q3$sim_gaussian_large.pve, 
                       dscout_Q3$score.hit, dscout_Q3$score.signal_num, dscout_Q3$score.cs_medianSize,
                       dscout_Q3$score.top_hit, dscout_Q3$sim_gaussian_large.mean_corX, dscout_Q3$susie_large.avg_purity)
names(dscout_df) = c('effect_num', 'pve','hit', 'cs_num', 'cs_size', 'top_hit', 'corX', 'avg_purity')

In [3]:
hitsum.summary = aggregate(hit ~ effect_num + pve, dscout_df, sum)
hitmean.summary = aggregate(hit ~ effect_num + pve, dscout_df, mean)
power.summary = hitmean.summary
power.summary$power = hitmean.summary$hit / power.summary$effect_num
fdr.summary = aggregate(cs_num ~ effect_num + pve, dscout_df, sum)
fdr.summary$fdr = round(1 - hitsum.summary$hit / fdr.summary$cs_num, 4)
meannonzero = function(x){mean(x[x!=0])}
setsize.summary = aggregate(cs_size ~ effect_num + pve, dscout_df, meannonzero)
tophit.summary = aggregate(top_hit ~ effect_num + pve, dscout_df, sum)
tophit.summary$tophit_rate = round(tophit.summary$top_hit / fdr.summary$cs_num , 4)
corX.summary = aggregate(corX ~ effect_num + pve, dscout_df, mean)
cs_num.summary = aggregate(cs_num ~ effect_num + pve, dscout_df, meannonzero)
dscout_df$avg_purity[is.na(dscout_df$avg_purity)]=0
purity.summary = aggregate(avg_purity ~ effect_num + pve, dscout_df, meannonzero)

In [4]:
dscout.summary = data.frame(power.summary$effect_num, power.summary$pve, corX.summary$corX,
                            power.summary$power, fdr.summary$fdr, setsize.summary$cs_size, 
                            cs_num.summary$cs_num, tophit.summary$tophit_rate, purity.summary$avg_purity)
names(dscout.summary) = c('effect_num', 'pve', 'mean_corX','power', 
                          'fdr', 'cs_size', 'cs_num','top_hit_rate', 'avg_purity')

In [5]:
is.nan.data.frame <- function(x)
do.call(cbind, lapply(x, is.nan))
dscout.summary[is.nan(dscout.summary)] = 0